In [ ]:
#path = '/content/drive/My Drive/dax_ml/orgdat_ab240409_size/cnv_org2trd/train_data/'
path = '/content/'
triggerCol = "mytrig_hi"   #"trigger500lo"  mytrig_hi

In [ ]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
import plotly.express as px
import plotly.graph_objs as go
import os

In [ ]:
# Funktion zum Erstellen von Datensätzen für LSTM aus einem Dataset , die gewählte tiefe der vergangenheit ist 10
# das übergebene dataset hat das format: anzahlBlöcke x ( [list der features(20)] + trigger )
def create_trainingsset(dataset, look_back=10):
  X, Y = [], []
  for i in range(len(dataset)-look_back-1):
    a = dataset[i:(i+look_back), :-1]   # array ohne letzte spalte
    X.append(a)
    Y.append(dataset[i + look_back, -1])  # letzte spalte ist Y
  return np.array(X), np.array(Y)

In [ ]:
  def create_day4lstm(path, name, triggercol_):
    df3  = pd.read_csv( path + name )
    mytrig = True if (triggercol_[:6] == "mytrig") else False
    if mytrig:
      dff  = pd.read_csv( path + '../fix/' + name[:-7] +'Scorefix1.csv')
      df3c = pd.concat([df3, dff] , axis = 1)
    else:
      df3c = df3

    df3c["avg1000diff250"] =df3c["avg1000diffAct"] - df3c["avg250diffAct"]
    df3c["ago250diffAct"]  =df3c["ago250price"]   -df3c["priceMean"]
    df3c["ago1000diffAct"] =df3c["ago1000price"]  -df3c["priceMean"]
    df3c["trigger500hi"]   =df3c['trigger500'] .shift(+2, fill_value=0).clip(lower=0)
    df3c["trigger500lo"]   =df3c['trigger500'] .shift(+2, fill_value=0).clip(upper=0) * -1
    df3c["trigger1000hi"]  =df3c['trigger1000'].shift(+2, fill_value=0).clip(lower=0)
    df3c["trigger1000lo"]  =df3c['trigger1000'].shift(+2, fill_value=0).clip(upper=0) * -1
    if mytrig:
      df3c["mytrig_hi"]  =df3c['mytrig_'].shift(+2, fill_value=0).clip(lower=0)         # shift(+1
      df3c["mytrig_lo"]  =df3c['mytrig_'].shift(+2, fill_value=0).clip(upper=0) * -1

    #   scoresize = 25 : anzahl trades als kleinste auswertungsblock mit min, max, mean, etc...  (zum traden evtl. zu groß gewaehlt)
    #   avgsize   = 50 : anzahl trades die für avgXX werte genommenw werden
    #   alle price angaben sind relativ zum startprice des tages
    featr  = [  "timescore",      #  je kürzer der block desto wertvoller deshalb (3/timediff_sec)
                "volSum",         #  summe gehandelte trades im Block
                "prcdiff",        #  aenderung des Preises zum vorghrigen block (prc_last)
                "prcgrad",        #  Steigung der Preisaenderung (je steiler, desto wertvoller)
                "priceMean0",     #  avererage der letzten avgsize trades

                "ago250diffAct",
                "ago250diff",     #  aenderung des Preises vor 250 Bloecken zum vorherigen block (prc_last)
                "ago250grad",     #  steigung des Preises vor 250 Bloecken zum vorherigen block (prc_last)
                "avg250diff",
                "avg250grad",
                "avg250diffAct",  #  differenz des 250-Bloecke Avg.Preises zum actual Price
                "avg250crossAct", #  Schneiden der 250 line und der Act Line (-1 von unten nach oben, 1 von oben nqch unten)

                "ago1000diffAct",
                "ago1000diff",    #  dto.fuer 1000 Bloecke
                "ago1000grad",
                "avg1000diff",
                "avg1000grad",
                "avg1000diffAct",
                "avg1000crossAct",

                "avg1000diff250"]

    np_in = df3c[ featr ].to_numpy()[50:]
    np_trans = preprocessing.StandardScaler().fit_transform(np_in)
    data = np.column_stack((np_trans, df3c[ triggercol_ ].to_numpy()[50:]))

    train_size = int(len(data) * 0.95)
    train_data = data[:train_size]
    test_data  = data[train_size:]

    look_back = 10
    x_train, y_train = create_dataset(train_data, look_back)
    return x_train, y_train

In [ ]:
ii=0
for name in os.listdir(path + 'train/'):
  if name.endswith(".csv"):
    X_train2, Y_train2 = create_day4lstm(path+'train/', name, triggerCol)
    X_train_all = X_train2 if ii == 0 else np.concatenate((X_train_all, X_train2), axis=0)
    Y_train_all = Y_train2 if ii == 0 else np.concatenate((Y_train_all, Y_train2), axis=0)
    print(name)
    ii = ii+1

print(ii)

In [ ]:
# 2. LSTM-Modell erstellen
model = tf.keras.Sequential()
model.add(tf.keras.layers.LSTM(units=32, return_sequences=True, input_shape=(X_train_all.shape[1], X_train_all.shape[2])))
model.add(tf.keras.layers.LSTM(units=32))
model.add(tf.keras.layers.Dense(1, activation= "sigmoid"))           # dense(1 ... )"sigmoid"))
# 3. Modell kompilieren und trainieren
model.compile(optimizer='adam',  loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),    metrics= ['accuracy'])
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                          │ (None, 10, 32)              │           6,784 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 32)                  │           8,320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 15,137 (59.13 KB)

 Trainable params: 15,137 (59.13 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# early_stopping = EarlyStopping( monitor='loss', baseline=0.0000000001 )   #klappt nicht richtig
checkpoint = ModelCheckpoint( filepath='/content/model/chkmodel_' + triggerCol + '83_best9.keras', monitor='loss', save_best_only=True)

model.fit( X_train_all, Y_train_all, epochs=97, batch_size=4, verbose=2, callbacks=[checkpoint] )

In [ ]:
#model.save(path + 'model/model_' + triggerCol + '_1.keras')
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                          │ (None, 10, 32)              │           6,784 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 32)                  │           8,320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 45,413 (177.40 KB)

 Trainable params: 15,137 (59.13 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 30,276 (118.27 KB)